# 블로그 설명

해당 자료에 대한 설명은 아래 블로그에도 올려두었습니다.
- https://lsjsj92.tistory.com/568

----

# 추천 시스템 - 아이템 기반 협업 필터링(item based collaborative filtering)

이번 글은 추천 시스템에서 아이템 기반 협업 필터링에 관해 작성합니다.  
해당 자료는 아래에서 참고했습니다.

- https://www.kaggle.com/manisha1409raj/recommender-system-collaborative-filtering
- https://medium.com/sfu-big-data/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0
- https://www.coursera.org/learn/collaborative-filtering/
- https://www.kaggle.com/rounakbanik/movie-recommender-systems
- https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system


데이터는 kaggle의 **The movies Dataset (https://www.kaggle.com/rounakbanik/the-movies-dataset)** 을 사용했습니다.

# 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('ratings_small.csv')

In [3]:
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


지금은 userId 컬럼과 movieId 컬럼이 따로따로 존재합니다.   
아이템 기반 협업 필터링(item based collaborative filtering) 기반으로 추천 시스템을 만드려면 user-item 테이블로 만들어주어야 합니다.

pivot_table을 이용해서 만들어주겠습니다.

In [4]:
data = data.pivot_table('rating', index = 'userId', columns = 'movieId')

In [5]:
data.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.shape

(671, 9066)

그러면 위와 같이 사용자 아이디 별 영화에 대한 평점을 볼 수 있는 테이블이 만들어지게 됩니다.

근데 문제는 여기서 영화 아이디는 있지만 영화 title이 없다는 문제가 있습니다. 영화 title을 가져와서 merge하겠습니다.

In [7]:
ratings = pd.read_csv('ratings_small.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')


In [8]:
movies.rename(columns = {'id': 'movieId'}, inplace = True)

In [9]:
ratings_movies = pd.merge(ratings, movies, on = 'movieId')

In [10]:
ratings_movies.head(1)

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296


In [11]:
ratings_movies.shape

(18571, 23)

In [12]:
data = ratings_movies.pivot_table('rating', index = 'userId', columns = 'title').fillna(0)

In [13]:
data.head()

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
data.shape

(670, 856)

그러면 위와 같이 사용자별 영화 title에 따라 평점을 매긴 정보를 볼 수 있게 됩니다.

근데 문제가 있습니다. 아이템 기반 협업 필터링이므로 row가 user 기반이면 안됩니다.  
item이 row가 되어야 합니다. 왜냐하면 cosine similarity를 구할 때 row 기반으로 유사도를 측정하기 때문이죠.  

그래서 row를 item으로 변경합니다.

In [15]:
data = data.transpose()
data.head(2)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
data.shape

(856, 670)

이렇게 변경이 되었습니다.

이제 item별로 유사한 것을 측정합니다.

In [17]:
movie_sim = cosine_similarity(data, data)
print(movie_sim.shape)

(856, 856)


In [18]:
movie_sim_df = pd.DataFrame(data = movie_sim, index = data.index, columns = data.index)

In [19]:
movie_sim_df.head(3)

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,1.0,0.0,0.0,0.182153,0.0,0.022069,0.085323,0.0,0.0,0.10349,...,0.059856,0.0,0.161801,0.088076,0.0,0.0,0.097588,0.0,0.0,0.014121
12 Angry Men,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1408,0.0,0.0,1.0,0.447214,0.0,0.173381,0.028245,0.0,0.0,0.00000,...,0.146955,0.0,0.148968,0.140265,0.0,0.0,0.191675,0.0,0.0,0.000000


그러면 이제 특정 영화와 비교했을 때 그 영화와 유사한 영화들을 추천해주면 됩니다.

In [20]:
movie_sim_df["X-Men Origins: Wolverine"].sort_values(ascending=False)[1:10]

title
Romeo Must Die                        0.649625
The Wedding Planner                   0.631669
Dogtown and Z-Boys                    0.501189
An Unfinished Life                    0.485643
Conquest of the Planet of the Apes    0.474626
Reign Over Me                         0.458155
The Terminal                          0.445337
Young Frankenstein                    0.423840
Whale Rider                           0.394136
Name: X-Men Origins: Wolverine, dtype: float64

In [21]:
movie_sim_df["Harry Potter and the Half-Blood Prince"].sort_values(ascending=False)[1:10]

title
Once                                      1.000000
Rendition                                 1.000000
Liar Liar                                 1.000000
Harry Potter and the Half-Blood Prince    1.000000
The Blue Lagoon                           1.000000
Synecdoche, New York                      1.000000
The Astronaut Farmer                      0.970143
Schindler's List                          0.724286
The Last King of Scotland                 0.707107
Name: Harry Potter and the Half-Blood Prince, dtype: float64

In [22]:
movie_sim_df["Harry Potter and the Half-Blood Prince"].sort_values(ascending=False)[:10]

title
Family Plot                               1.000000
Once                                      1.000000
Rendition                                 1.000000
Liar Liar                                 1.000000
Harry Potter and the Half-Blood Prince    1.000000
The Blue Lagoon                           1.000000
Synecdoche, New York                      1.000000
The Astronaut Farmer                      0.970143
Schindler's List                          0.724286
The Last King of Scotland                 0.707107
Name: Harry Potter and the Half-Blood Prince, dtype: float64

In [23]:
movie_sim_df["King Kong"].sort_values(ascending=False)[1:10]

title
Fantasia                                  0.648886
2046                                      0.648886
Liar Liar                                 0.486664
Rendition                                 0.486664
Family Plot                               0.486664
Harry Potter and the Half-Blood Prince    0.486664
The Blue Lagoon                           0.486664
Once                                      0.486664
Synecdoche, New York                      0.486664
Name: King Kong, dtype: float64